In [ ]:
import os
import sys

os.chdir('..')

In [ ]:
import joblib

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # LDA (Linear Discriminant Analysis) or FDA (Fisher Discriminant Analysis)
# from sklearn.neural_network import MLPClassifier # ANN
# from sklearn.tree import DecisionTreeClassifier # CTA
# from sklearn.ensemble import RandomForestClassifier # RF
# from sklearn.ensemble import GradientBoostingClassifier # GBM
from xgboost import XGBClassifier # XGB
# from lightgbm import LGBMClassifier # LGBM
# from statsmodels.api import GLM # GLM
import statsmodels.api as sm
# from elapid import MaxentModel # MaxEnt
import elapid

from lib.cnn.models.dann import SDM_DNN3

In [ ]:
import numpy as np
import torch
import torch.nn as nn

from lib.dataset import EnvironmentalDataset
from lib.raster import PatchExtractor
from lib.utils import set_reproducibility, load_model_state

import rasterio
import rasterio.plot as rplt
import matplotlib.pyplot as plt

# For reproducibility
random_seed = 42
set_reproducibility(random_seed=random_seed)

# SETTINGS
# files
RASTER_PATH = './data/rasters_KR/'

# environmental patches
PATCH_SIZE = 1

# model params
DROPOUT = 0
N_LABELS = 1 # probability of presence

In [ ]:
# create patch extractor and add all default rasters
extractor = PatchExtractor(RASTER_PATH, size=PATCH_SIZE, verbose=True)
extractor.add_all(normalized=True, transform=None, ignore=[])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
"""
get map_dataset for predicting habitat suitability maps
"""

# We want to make predictions of total grid cells.
# Get all locations of raster grid cells without cells of no data.
# For this, it needs to use raster with the highest resolution (bioclim: ~900m, globcover: 300m, SRTM: ~90m).
# SRTM_elevation에 nodata가 없어서 유효한 픽셀만 고르기가 불가능.
# 그리고, habitat suitability map을 반드시 높은 해상도를 가진 래스터 데이터를 사용하란 법은 없는듯.
# PL 접근법과도 문제될 점은 없어보임.

# raster = extractor.get_raster(raster_name='srtm_elevation') # (6937, 6303)
# raster = extractor.get_raster(raster_name='bioclim_1') # (694, 630)
valid_pos = extractor.get_valid_positions(raster_name='bioclim_1')
print(valid_pos.shape)
temp_labels = np.zeros(len(valid_pos), dtype=np.float32)
temp_ids = np.arange(len(valid_pos))

validmap_dataset = EnvironmentalDataset(temp_labels, valid_pos, temp_ids, extractor)
validmap_dataset

In [ ]:
# LOAD ML Models 

folder = './pretrained/황소개구리_final'
# folder = './pretrained/오목눈이_final'
# folder = './pretrained/알락할미새_final'

# folder = './pretrained/오목눈이_과제'
# folder = './pretrained/오목눈이_과제2'

# MODEL_NAMES = ['sdm_ann.pkl', 'sdm_gbm.pkl', 'sdm_lgbm.pkl', 'sdm_rf.pkl', 'sdm_xgb.pkl', 'sdm_dnn_best.pt', 'sdm_dnn_pl_best.pt']
# MODEL_NAMES = ['sdm_ann.pkl', 'sdm_rf.pkl', 'sdm_gbm.pkl', 'sdm_xgb.pkl', 'sdm_lgbm.pkl', 'sdm_maxent.ela']

# MODEL_NAMES = ['sdm_dnn_5.pt'] # 황소개구리
# MODEL_NAMES = ['sdm_dnn_3.pt'] # 오목눈이
# MODEL_NAMES = ['sdm_dnn_11.pt'] # 알락할미새

MODEL_NAMES = ['sdm_dnn_pl_best.pt']


for model_name in MODEL_NAMES:
    X_set, _ = validmap_dataset.numpy()

    path = f'{folder}/{model_name}'
    if model_name == 'sdm_xgb.pkl': # xgb
        model = XGBClassifier(eval_metric='logloss', random_state=random_seed)
        model.load_model(path)
        predictions = model.predict_proba(X_set)[:, 1]

    elif model_name.split('.')[-1] == 'pt': # dnn
        model = SDM_DNN3(in_features=extractor.n_data_dims, n_labels=N_LABELS, drop_out=DROPOUT, activation_func=nn.SELU).to(device)
        load_model_state(model, path)
        model.eval()
        predictions = model(torch.from_numpy(X_set).to(device)).detach().cpu().numpy()

    elif model_name == 'sdm_glm.pkl': # glm
        model = sm.load(path)
        predictions = model.predict(X_set)
    
    elif model_name == 'sdm_maxent.ela':
        model = elapid.load_object(path)
        predictions = model.predict(X_set)

    else: # ann, gbm, lgbm, rf
        model = joblib.load(path)
        predictions = model.predict_proba(X_set)[:, 1]


    """
    generates habitat suitability maps (only predictable area)
    """

    # make new raster map
    raster = extractor.get_raster('bioclim_1')
    new_raster = np.zeros(shape=raster.shape, dtype=np.float32)

    for idx, (lat, long) in enumerate(valid_pos):
        row, col = raster.coords_to_index(long, lat)
        new_raster[row, col] = predictions[idx]

    """
    raster 데이터 생성
    https://rasterio.readthedocs.io/en/latest/topics/writing.html
    """

    #image_folder = f'./images/{}'
    #os.makedirs(image_folder, exist_ok=True)
    image_path = f'./images/HSM_{os.path.basename(folder)}_{os.path.splitext(model_name)[0]}.tif'

    with rasterio.Env():
        src = rasterio.open('./data/rasters_KR/bioclim_1/bioclim_1.tif')
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,
            count=1, # band count
            compress='lzw')
        
        with rasterio.open(image_path, 'w', **profile) as dst:
            dst.write(new_raster.astype(rasterio.float32), 1)

    r = rasterio.open(image_path)
    fig1, ax1 = plt.subplots(figsize=(10,10))
    rplt.show(r, ax=ax1, cmap='viridis')